# 🎡 Adventure World Theme Park Simulation
## COMP1005 Assignment - Final Version

**Matches reference video exactly with proper patron movement!**

### Features:
- ✅ Ferris Wheel with rotating gondolas
- ✅ Pirate Ship with swinging motion
- ✅ Bumper Cars arena
- ✅ Roller Coaster with vertical track
- ✅ Patrons moving naturally around the park
- ✅ Queue visualization
- ✅ Real-time statistics

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
from collections import deque
from IPython.display import HTML
from matplotlib import rc

rc('animation', html='jshtml')
print("✅ Libraries imported successfully!")

In [ ]:
# Base Ride Class
class Ride:
    def __init__(self, x, y, width, height, capacity, duration, name):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.capacity = capacity
        self.duration = duration
        self.name = name
        self.state = "IDLE"
        self.queue = deque()
        self.riders = []
        self.time_counter = 0
        self.total_riders = 0
        self.angle = 0
        self.queue_position = (x, y - height/2 - 3)

    def get_bounding_box(self):
        half_w = self.width / 2
        half_h = self.height / 2
        return (self.x - half_w, self.x + half_w, self.y - half_h, self.y + half_h)

    def is_in_bounds(self, px, py):
        x_min, x_max, y_min, y_max = self.get_bounding_box()
        padding = 5
        return (x_min - padding) <= px <= (x_max + padding) and (y_min - padding) <= py <= (y_max + padding)

    def overlaps(self, other_ride):
        x1_min, x1_max, y1_min, y1_max = self.get_bounding_box()
        x2_min, x2_max, y2_min, y2_max = other_ride.get_bounding_box()
        return not (x1_max < x2_min or x1_min > x2_max or y1_max < y2_min or y1_min > y2_max)

    def add_to_queue(self, patron):
        self.queue.append(patron)
        patron.state = "QUEUING"
        patron.target_ride = self

    def step_change(self):
        # ALWAYS animate rides - this makes them continuously move!
        self.time_counter += 1
        self.angle = self._calculate_angle()
        
        if self.state == "IDLE" and len(self.queue) > 0:
            self.state = "LOADING"
        elif self.state == "LOADING":
            while len(self.riders) < self.capacity and len(self.queue) > 0:
                patron = self.queue.popleft()
                self.riders.append(patron)
                patron.state = "RIDING"
                self.total_riders += 1
            if len(self.riders) > 0:
                self.state = "RUNNING"
                self.time_counter = 0
        elif self.state == "RUNNING":
            if self.time_counter >= self.duration:
                for rider in self.riders:
                    rider.state = "ROAMING"
                    rider.target_ride = None
                self.riders = []
                self.state = "IDLE"
                self.time_counter = 0

    def _calculate_angle(self):
        return 0

    def plot_queue(self, ax):
        qx, qy = self.queue_position
        for i, patron in enumerate(list(self.queue)[:8]):
            square = patches.Rectangle((qx + i*2.5 - 10, qy - 1.5), 2, 2,
                                       facecolor=patron.color, edgecolor='black', linewidth=0.5)
            ax.add_patch(square)

print("✅ Ride base class ready - RIDES ALWAYS ANIMATE NOW!")

In [ ]:
# Ferris Wheel
class FerrisWheel(Ride):
    def __init__(self, x, y, radius, capacity, duration, name="Ferris Wheel"):
        super().__init__(x, y, radius*2.5, radius*2.5, capacity, duration, name)
        self.radius = radius
        self.num_gondolas = 8

    def _calculate_angle(self):
        return (self.time_counter * 360 / self.duration) % 360

    def plot(self, ax):
        circle = patches.Circle((self.x, self.y), self.radius, linewidth=2,
                               edgecolor='darkslateblue', facecolor='none', zorder=3)
        ax.add_patch(circle)
        hub = patches.Circle((self.x, self.y), self.radius*0.1, linewidth=1.5,
                            edgecolor='black', facecolor='gold', zorder=5)
        ax.add_patch(hub)
        
        for i in range(self.num_gondolas):
            angle = np.radians(i * (360/self.num_gondolas) + self.angle)
            x_end = self.x + self.radius * np.cos(angle)
            y_end = self.y + self.radius * np.sin(angle)
            ax.plot([self.x, x_end], [self.y, y_end], 'darkslateblue', linewidth=1.5, zorder=3)
            gondola = patches.Circle((x_end, y_end), 2.5, facecolor='cornflowerblue',
                                    edgecolor='darkblue', linewidth=1, zorder=4)
            ax.add_patch(gondola)
        
        platform_width = self.radius * 0.6
        platform_height = 5
        platform_y = self.y - self.radius - platform_height
        platform = patches.Rectangle((self.x - platform_width/2, platform_y),
                                     platform_width, platform_height, facecolor='gray',
                                     edgecolor='darkgray', linewidth=1.5, zorder=2)
        ax.add_patch(platform)
        self.plot_queue(ax)

# Pirate Ship
class PirateShip(Ride):
    def __init__(self, x, y, length, capacity, duration, name="Pirate Ship"):
        super().__init__(x, y, length*1.2, length*1.2, capacity, duration, name)
        self.length = length
        self.max_angle = 50

    def _calculate_angle(self):
        progress = self.time_counter / self.duration
        return self.max_angle * np.sin(2 * np.pi * progress * 2)

    def plot(self, ax):
        base_width = self.length * 0.8
        base_height = 5
        base_y = self.y - self.length * 0.5
        platform = patches.Rectangle((self.x - base_width/2, base_y - base_height),
                                     base_width, base_height, facecolor='gray',
                                     edgecolor='darkgray', linewidth=1.5, zorder=2)
        ax.add_patch(platform)
        
        height = self.length * 0.8
        triangle = np.array([[self.x - base_width/2, base_y],
                            [self.x + base_width/2, base_y],
                            [self.x, base_y + height]])
        support = patches.Polygon(triangle, closed=True, edgecolor='gray',
                                 facecolor='none', linewidth=2, zorder=3)
        ax.add_patch(support)
        
        pivot_y = base_y + height * 0.7
        ax.plot(self.x, pivot_y, 'ko', markersize=6, zorder=5)
        
        angle_rad = np.radians(self.angle)
        ship_x = self.x + (self.length/2.2) * np.sin(angle_rad)
        ship_y = pivot_y - (self.length/2.2) * np.cos(angle_rad)
        ax.plot([self.x, ship_x], [pivot_y, ship_y], 'gray', linewidth=4, zorder=3)
        
        boat_length, boat_height = 10, 5
        boat_points = np.array([[-boat_length/2, -boat_height/2],
                               [boat_length/2, -boat_height/2],
                               [boat_length/2 - 1.5, boat_height/2],
                               [-boat_length/2 + 1.5, boat_height/2]])
        cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
        rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
        rotated_boat = boat_points @ rotation_matrix.T
        rotated_boat[:, 0] += ship_x
        rotated_boat[:, 1] += ship_y
        boat = patches.Polygon(rotated_boat, closed=True, facecolor='royalblue',
                              edgecolor='darkblue', linewidth=2, zorder=4)
        ax.add_patch(boat)
        self.plot_queue(ax)

# Bumper Cars - NOW WITH ALWAYS-MOVING CARS!
class BumperCars(Ride):
    def __init__(self, x, y, width, height, capacity, duration, name="Bumper Cars"):
        super().__init__(x, y, width, height, capacity, duration, name)
        # Default car colors that show when arena is empty
        self.default_car_colors = ['red', 'blue', 'green', 'yellow', 'purple', 'cyan']

    def _calculate_angle(self):
        return (self.time_counter * 10) % 360

    def plot(self, ax):
        x_min, x_max, y_min, y_max = self.get_bounding_box()
        outer = patches.Rectangle((x_min - 3, y_min - 3), self.width + 6, self.height + 6,
                                  linewidth=2.5, edgecolor='gray', facecolor='lightgray',
                                  alpha=0.4, zorder=2)
        ax.add_patch(outer)
        middle = patches.Rectangle((x_min - 1.5, y_min - 1.5), self.width + 3, self.height + 3,
                                   linewidth=2, edgecolor='slategray', facecolor='none', zorder=2)
        ax.add_patch(middle)
        arena = patches.Rectangle((x_min, y_min), self.width, self.height, linewidth=2.5,
                                 edgecolor='darkslategray', facecolor='#E8F5C8', alpha=0.8, zorder=2)
        ax.add_patch(arena)
        
        if self.state == "IDLE":
            center = patches.Circle((self.x, self.y), 3, facecolor='orange',
                                   edgecolor='darkorange', linewidth=1.5, zorder=3)
            ax.add_patch(center)
        
        # ALWAYS show moving cars!
        if len(self.riders) > 0:
            # Show actual riders as cars
            for i, rider in enumerate(self.riders):
                angle = (self.angle + i * 360/len(self.riders)) % 360
                offset_x = self.width * 0.3 * np.cos(np.radians(angle))
                offset_y = self.height * 0.3 * np.sin(np.radians(angle))
                car = patches.Circle((self.x + offset_x, self.y + offset_y), 2,
                                    facecolor=rider.color, edgecolor='black',
                                    linewidth=1, zorder=4)
                ax.add_patch(car)
        else:
            # Show default cars when empty - ALWAYS MOVING!
            num_default_cars = len(self.default_car_colors)
            for i in range(num_default_cars):
                angle = (self.angle + i * 360/num_default_cars) % 360
                offset_x = self.width * 0.3 * np.cos(np.radians(angle))
                offset_y = self.height * 0.3 * np.sin(np.radians(angle))
                car = patches.Circle((self.x + offset_x, self.y + offset_y), 2,
                                    facecolor=self.default_car_colors[i],
                                    edgecolor='black', linewidth=1, zorder=4)
                ax.add_patch(car)
        
        self.plot_queue(ax)

# Roller Coaster - NOW WITH ALWAYS-MOVING CAR!
class RollerCoaster(Ride):
    def __init__(self, x, y, width, height, capacity, duration, name="Roller Coaster"):
        super().__init__(x, y, width, height, capacity, duration, name)

    def _calculate_angle(self):
        return (self.time_counter * 720 / self.duration) % 360

    def plot(self, ax):
        x_min, x_max, y_min, y_max = self.get_bounding_box()
        platform = patches.Rectangle((x_min - 2, y_max - 4), self.width + 4, 4,
                                     facecolor='gold', edgecolor='darkorange',
                                     linewidth=2, zorder=3)
        ax.add_patch(platform)
        
        track_width = 8
        track = patches.Rectangle((self.x - track_width/2, y_min), track_width, self.height - 4,
                                  facecolor='slategray', edgecolor='darkgray',
                                  linewidth=2, zorder=2)
        ax.add_patch(track)
        ax.plot([self.x, self.x], [y_min, y_max - 4], 'white', linewidth=2,
               linestyle='--', alpha=0.8, zorder=2)
        
        for side in [-1, 1]:
            for i in range(3):
                cube_x = self.x + side * (track_width/2 + 5)
                cube_y = self.y + (i - 1) * 7
                cube = patches.Rectangle((cube_x - 2, cube_y - 2), 4, 4,
                                        facecolor='mediumseagreen', edgecolor='darkgreen',
                                        linewidth=1.5, zorder=3)
                ax.add_patch(cube)
        
        base = patches.Rectangle((x_min - 2, y_min - 5), self.width + 4, 5,
                                facecolor='gray', edgecolor='darkgray',
                                linewidth=1.5, zorder=2)
        ax.add_patch(base)
        
        # Always show the car moving up and down - even when empty!
        progress = (self.time_counter % self.duration) / self.duration
        car_y = y_min + (self.height - 10) * (0.5 + 0.5 * np.sin(2 * np.pi * progress))
        car = patches.Rectangle((self.x - 4, car_y - 3), 8, 6, facecolor='red',
                                edgecolor='darkred', linewidth=2, zorder=5)
        ax.add_patch(car)
        
        self.plot_queue(ax)

print("✅ All ride classes ready - ALL RIDES ALWAYS ANIMATE NOW!")

In [ ]:
# Patron Class with IMPROVED MOVEMENT
class Patron:
    def __init__(self, x, y, name, park):
        self.x = x
        self.y = y
        self.name = name
        self.park = park
        self.state = "ROAMING"
        self.target_ride = None
        self.target_x = None
        self.target_y = None
        self.speed = 2.0  # FASTER MOVEMENT!
        self.patience = 0
        self.max_patience = 200
        self.rides_taken = 0
        self.color = np.random.choice(['red', 'blue', 'green', 'purple',
                                       'magenta', 'cyan', 'brown', 'pink',
                                       'orange', 'lime', 'navy', 'maroon'])
        self.frozen_time = 5
        self.stuck_counter = 0

    def step_change(self, timestep):
        if timestep < self.frozen_time:
            return
        self.patience += 1
        if self.state == "ROAMING":
            self._roam()
        elif self.state == "LEAVING":
            self._leave_park()
        if self.patience > self.max_patience and self.state == "ROAMING":
            self.state = "LEAVING"

    def _roam(self):
        # Target-based movement!
        if self.target_ride is None and np.random.random() < 0.02:
            available_rides = [r for r in self.park.rides if len(r.queue) < 8]
            if available_rides:
                self.target_ride = np.random.choice(available_rides)
                self.target_x = self.target_ride.x
                self.target_y = self.target_ride.y

        if self.target_ride is not None:
            dx = self.target_x - self.x
            dy = self.target_y - self.y
            dist = np.sqrt(dx**2 + dy**2)
            if dist < 15:
                self.target_ride.add_to_queue(self)
                self.rides_taken += 1
                self.patience = 0
                self.target_ride = None
                self.target_x = None
                self.target_y = None
                return
            if dist > 0:
                new_x = self.x + self.speed * (dx / dist)
                new_y = self.y + self.speed * (dy / dist)
                if self.park.is_valid_position(new_x, new_y):
                    self.x = new_x
                    self.y = new_y
                    self.stuck_counter = 0
                else:
                    self.stuck_counter += 1
                    if self.stuck_counter > 10:
                        self.target_ride = None
                        self.target_x = None
                        self.target_y = None
                        self.stuck_counter = 0
        else:
            if self.target_x is None or np.random.random() < 0.05:
                self.target_x = np.random.uniform(20, self.park.width - 20)
                self.target_y = np.random.uniform(20, self.park.height - 20)
            
            dx = self.target_x - self.x
            dy = self.target_y - self.y
            dist = np.sqrt(dx**2 + dy**2)
            if dist < 5:
                self.target_x = None
                self.target_y = None
            elif dist > 0:
                new_x = self.x + self.speed * (dx / dist)
                new_y = self.y + self.speed * (dy / dist)
                if self.park.is_valid_position(new_x, new_y):
                    self.x = new_x
                    self.y = new_y
                else:
                    self.target_x = None
                    self.target_y = None

    def _leave_park(self):
        exit_x, exit_y = self.park.get_nearest_exit(self.x, self.y)
        dx = exit_x - self.x
        dy = exit_y - self.y
        dist = np.sqrt(dx**2 + dy**2)
        if dist < 3:
            self.park.remove_patron(self)
        else:
            self.x += self.speed * (dx / dist)
            self.y += self.speed * (dy / dist)

    def plot(self, ax):
        if self.state in ["ROAMING", "LEAVING"]:
            ax.plot(self.x, self.y, 'o', color=self.color, markersize=7,
                   markeredgecolor='black', markeredgewidth=0.5, zorder=10)

print("✅ Patron class ready with improved movement!")

In [ ]:
# ThemePark Class
class ThemePark:
    def __init__(self, width=200, height=200):
        self.width, self.height = width, height
        self.rides = []
        self.patrons = []
        self.exits = [(10, height/2), (width-10, height/2)]
        self.timestep = 0
        self.patron_counter = 0
        self.total_patrons_entered = 0
        self.total_patrons_left = 0
        self.stats_history = {'timestep': [], 'num_patrons': [],
                             'num_queuing': [], 'num_riding': [], 'total_rides': []}

    def add_ride(self, ride):
        for existing_ride in self.rides:
            if ride.overlaps(existing_ride):
                return False
        self.rides.append(ride)
        return True

    def add_patron(self):
        entry_x, entry_y = self.exits[np.random.randint(len(self.exits))]
        entry_x += np.random.uniform(-5, 5)
        entry_y += np.random.uniform(-10, 10)
        patron = Patron(entry_x, entry_y, f"P{self.patron_counter}", self)
        self.patron_counter += 1
        self.patrons.append(patron)
        self.total_patrons_entered += 1

    def remove_patron(self, patron):
        if patron in self.patrons:
            self.patrons.remove(patron)
            self.total_patrons_left += 1

    def is_valid_position(self, x, y):
        if x < 5 or x > self.width - 5 or y < 5 or y > self.height - 5:
            return False
        for ride in self.rides:
            if ride.is_in_bounds(x, y):
                return False
        return True

    def get_nearest_exit(self, x, y):
        return min(self.exits, key=lambda e: np.sqrt((e[0]-x)**2 + (e[1]-y)**2))

    def step(self):
        self.timestep += 1
        if np.random.random() < 0.2 and len(self.patrons) < 30:  # Higher spawn rate!
            self.add_patron()
        for ride in self.rides:
            ride.step_change()
        for patron in list(self.patrons):
            patron.step_change(self.timestep)
        self._update_statistics()

    def _update_statistics(self):
        num_queuing = sum(1 for p in self.patrons if p.state == "QUEUING")
        num_riding = sum(1 for p in self.patrons if p.state == "RIDING")
        total_rides = sum(r.total_riders for r in self.rides)
        self.stats_history['timestep'].append(self.timestep)
        self.stats_history['num_patrons'].append(len(self.patrons))
        self.stats_history['num_queuing'].append(num_queuing)
        self.stats_history['num_riding'].append(num_riding)
        self.stats_history['total_rides'].append(total_rides)

    def plot(self, ax):
        ax.clear()
        ax.set_xlim(0, self.width)
        ax.set_ylim(0, self.height)
        ax.set_aspect('equal')
        ax.set_title('Theme Park', fontsize=14, fontweight='bold')
        ax.set_facecolor('lightgreen')
        
        pathway_h = patches.Rectangle((0, self.height/2 - 10), self.width, 20,
                                     facecolor='#C8C8C8', edgecolor='none', zorder=1)
        ax.add_patch(pathway_h)
        pathway_v = patches.Rectangle((self.width/2 - 10, 0), 20, self.height,
                                     facecolor='#C8C8C8', edgecolor='none', zorder=1)
        ax.add_patch(pathway_v)
        
        for exit_x, exit_y in self.exits:
            exit_marker = patches.Rectangle((exit_x-3, exit_y-3), 6, 6,
                                           facecolor='gray', edgecolor='darkgray',
                                           linewidth=1.5, zorder=2)
            ax.add_patch(exit_marker)
        
        for ride in self.rides:
            ride.plot(ax)
        for patron in self.patrons:
            patron.plot(ax)

    def plot_with_statistics(self, fig, ax_park, ax_stats):
        self.plot(ax_park)
        ax_stats.clear()
        if len(self.stats_history['timestep']) > 1:
            ax_stats.plot(self.stats_history['timestep'], self.stats_history['num_patrons'],
                         label='Total Patrons', linewidth=2, color='blue')
            ax_stats.plot(self.stats_history['timestep'], self.stats_history['num_queuing'],
                         label='Queuing', linewidth=2, color='orange')
            ax_stats.plot(self.stats_history['timestep'], self.stats_history['num_riding'],
                         label='Riding', linewidth=2, color='green')
            ax_stats.set_xlabel('Timestep')
            ax_stats.set_ylabel('Number of Patrons')
            ax_stats.set_title('Park Statistics')
            ax_stats.legend(loc='upper left')
            ax_stats.grid(True, alpha=0.3)

print("✅ ThemePark class ready")

In [ ]:
# Create the park
park = ThemePark(width=200, height=200)

park.add_ride(FerrisWheel(50, 150, radius=20, capacity=8, duration=80, name="Ferris Wheel"))
park.add_ride(RollerCoaster(150, 150, width=20, height=60, capacity=6, duration=60, name="Tower Drop"))
park.add_ride(BumperCars(50, 50, width=40, height=35, capacity=8, duration=70, name="Bumper Cars"))
park.add_ride(PirateShip(150, 50, length=30, capacity=10, duration=50, name="Pirate Ship"))

for _ in range(10):
    park.add_patron()

print(f"🎡 Park ready with {len(park.rides)} rides and {len(park.patrons)} patrons!")
print("🚶 Patrons will move around the park with improved movement!")

In [ ]:
# Run the simulation!
MAX_TIMESTEPS = 400

fig = plt.figure(figsize=(16, 7))
ax_park = fig.add_subplot(121)
ax_stats = fig.add_subplot(122)

def update(frame):
    if frame < MAX_TIMESTEPS:
        park.step()
        park.plot_with_statistics(fig, ax_park, ax_stats)
    return ax_park, ax_stats

print("🎬 Starting animation...")
anim = FuncAnimation(fig, update, frames=MAX_TIMESTEPS, interval=50, blit=False, repeat=False)
plt.tight_layout()
HTML(anim.to_jshtml())

In [ ]:
# Final Statistics
print("="*60)
print("🎉 SIMULATION COMPLETE")
print("="*60)
print(f"⏱️  Total timesteps: {park.timestep}")
print(f"👥 Total patrons entered: {park.total_patrons_entered}")
print(f"🚪 Total patrons left: {park.total_patrons_left}")
print(f"🏞️  Patrons still in park: {len(park.patrons)}")
print(f"🎢 Total rides taken: {sum(r.total_riders for r in park.rides)}")
print("\n📊 Ride Statistics:")
for ride in park.rides:
    print(f"  🎡 {ride.name}: {ride.total_riders} riders, queue: {len(ride.queue)}, state: {ride.state}")